In [6]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "jax" or "torch"!

import keras_nlp
import tensorflow_datasets as tfds
import keras

import pandas as pd
import numpy as np
import tensorflow as tf

In [7]:
all_tabiturient_data = pd.read_json('all-reviews.jsonl', lines=True)

train_tabiturient_data=all_tabiturient_data[all_tabiturient_data['parent_url']!='https://tabiturient.ru/vuzu/mgtu']
test_tabiturient_data=all_tabiturient_data[all_tabiturient_data['parent_url']=='https://tabiturient.ru/vuzu/mgtu']

train_label_data = keras.utils.to_categorical(np.array(train_tabiturient_data['label']))
test_label_data = keras.utils.to_categorical(np.array(test_tabiturient_data['label']))

train_raw_label_data = np.array(train_tabiturient_data['label'])
test_raw_label_data = np.array(test_tabiturient_data['label'])

train_raw_text_data = np.array(train_tabiturient_data['content'])
test_raw_text_data = np.array(test_tabiturient_data['content'])

train_ds=tf.data.Dataset.from_tensor_slices((train_raw_text_data, train_label_data))
test_ds=tf.data.Dataset.from_tensor_slices((test_raw_text_data, test_label_data))

2024-05-31 11:15:36.650393: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


/home/vasilusa/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|                                                                                                                                                                                              | 0/1 [00:12<?, ? url/s]


Dl Completed...:   0%|                                                                                                                                                                                              | 0/1 [00:24<?, ? url/s]


Dl Completed...:   0%|                                                                                                                                                                                              | 0/1 [00:35<?, ? url/s]


Generating train examples...:  94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎        | 23587/25000 [00:09<00:00, 3531.88 examples/s]
                                                                                                                                                                                                                                            
Shuffling /home/vasilusa/tensorflow_datasets/imdb_reviews/plain_text/incomplete.A483CM_1.0.0/imdb_reviews-train.tfrecord*...:   0%|                                                                        | 0/25000 [00:00<?, ? examples/s]
Generating test examples...:  84%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                        | 21053/25000 [00:06<00:01, 3456.54 examples/s]


Generating test examples...:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 24818/25000 [00:07<00:00, 3550.90 examples/s]
                                                                                                                                                                                                                                            
Shuffling /home/vasilusa/tensorflow_datasets/imdb_reviews/plain_text/incomplete.A483CM_1.0.0/imdb_reviews-test.tfrecord*...:   0%|                                                                         | 0/25000 [00:00<?, ? examples/s]
Generating unsupervised examples...:  98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 49240/50000 [00:19<00:00, 3564.12 examples/s]
                                                    

Dataset imdb_reviews downloaded and prepared to /home/vasilusa/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_multi",
    num_classes=3,
)

17602216/17602216 [==============================] - 2s 0us/step


In [4]:
classifier.fit(x=train_raw_text_data, y=train_raw_label_data, batch_size=12, epochs=10)

2023-12-13 13:45:02.272766: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f69d4018c60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-13 13:45:02.272877: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-12-13 13:45:02.511345: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-13 13:45:02.842701: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:59] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. bert_classifier/bert_backbone/embeddings_dropout/dropout/random_uniform/RandomUniform
I0000 00:00:1702464308.197887    2278 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-12-1

1563/1563 [==============================] - 2766s 2s/step - loss: 0.4098 - sparse_categorical_accuracy: 0.8097 - val_loss: 0.3035 - val_sparse_categorical_accuracy: 0.8714


In [5]:
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible...",
  "I ll recommend this movie to all my friends"    
]

examples2 = ["What an amazing movie!", "A total waste of my time."]

In [ ]:
classifier.predict(examples)

In [18]:
classifier.save("bert_tiny_imdb.dat")

INFO:tensorflow:Assets written to: bert_tiny_imdb.dat/assets


INFO:tensorflow:Assets written to: bert_tiny_imdb.dat/assets


In [3]:
classifier2 = keras.saving.load_model("bert_tiny_imdb.dat")

In [6]:
classifier2.predict(examples2)

1/1 [==============================] - 1s 583ms/step


array([[0.02878699, 0.97121304],
       [0.9309194 , 0.06908057]], dtype=float32)